In [2]:
!pip install stable-baselines3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 958.1/958.1 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 55.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 31.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-c

In [4]:
!pip install shimmy

In [16]:
pip install gymnasium[all] stable-baselines3


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.4/374.4 kB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 60.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 39.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 59.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.4/243.4 kB 19.4 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for box2d-py
  Running setup.py clean for box2d-py
Failed to build box2d-py
ERROR: ERROR: Failed to build installable wheels for some pyproject

In [21]:
import gymnasium as gym  # ✅ Use Gymnasium instead of Gym
import numpy as np
from gymnasium import spaces
from stable_baselines3 import DQN
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.monitor import Monitor

In [22]:
# ✅ Custom Supply Chain Environment (Fixed Inheritance)
class SupplyChainEnv(gym.Env):  # ✅ Explicitly inherit from gym.Env
    def __init__(self):
        super(SupplyChainEnv, self).__init__()

        # ✅ Define Action Space: {0: Reduce Stock, 1: Maintain, 2: Increase Stock}
        self.action_space = spaces.Discrete(3)

        # ✅ Define Observation Space: Inventory level (bounded between 0 and 100)
        self.observation_space = spaces.Box(low=0, high=100, shape=(1,), dtype=np.float32)

        # ✅ Initial inventory level
        self.state = np.array([50], dtype=np.float32)

        # ✅ Maximum simulation steps
        self.max_steps = 50
        self.current_step = 0

    def step(self, action):
        """
        Take an action in the environment and update inventory state.
        """
        self.current_step += 1

        # Action Mapping:
        if action == 0:  # Reduce Stock
            self.state[0] -= 10
        elif action == 2:  # Increase Stock
            self.state[0] += 10

        # ✅ Demand Simulation: Randomized demand between 5 and 15 units
        demand = np.random.randint(5, 15)
        self.state[0] -= demand  # Deduct demand from stock
        self.state[0] = np.clip(self.state[0], 0, 100)  # Ensure inventory stays within bounds

        # ✅ Reward Function: Encourage inventory level close to 50
        reward = -abs(50 - self.state[0])  # Best reward when inventory ≈ 50

        # ✅ Termination Condition
        done = self.current_step >= self.max_steps

        return self.state, reward, done, False, {}  # ✅ Follow Gymnasium's API

    def reset(self, seed=None, options=None):
        """
        Reset the environment to the initial state.
        """
        self.state = np.array([50], dtype=np.float32)
        self.current_step = 0
        return self.state, {}  # ✅ Must return (obs, info) in Gymnasium

    def render(self):
        """
        Render the environment (prints inventory level).
        """
        print(f"Step: {self.current_step}, Inventory Level: {self.state[0]}")


In [23]:
# ✅ Create Environment & Wrap it Properly (Fixed Wrapping Issue)
env = SupplyChainEnv()
vec_env = DummyVecEnv([lambda: Monitor(env, filename=None)])  # ✅ Prevent logging errors

In [24]:
# ✅ Train DQN Model
model = DQN("MlpPolicy", vec_env, verbose=1, learning_rate=0.001, batch_size=32)
model.learn(total_timesteps=5000)

Using cpu device
-----------------------------------
| rollout/            |           |
|    ep_len_mean      | 50        |
|    ep_rew_mean      | -2.38e+03 |
|    exploration_rate | 0.62      |
| time/               |           |
|    episodes         | 4         |
|    fps              | 1143      |
|    time_elapsed     | 0         |
|    total_timesteps  | 200       |
| train/              |           |
|    learning_rate    | 0.001     |
|    loss             | 46.3      |
|    n_updates        | 24        |
-----------------------------------
-----------------------------------
| rollout/            |           |
|    ep_len_mean      | 50        |
|    ep_rew_mean      | -2.34e+03 |
|    exploration_rate | 0.24      |
| time/               |           |
|    episodes         | 8         |
|    fps              | 1097      |
|    time_elapsed     | 0         |
|    total_timesteps  | 400       |
| train/              |           |
|    learning_rate    | 0.001     |
|    loss  

In [25]:
# ✅ Save the trained model
model.save("supply_chain_dqn")

In [26]:
# ✅ Load and Test the Model
model = DQN.load("supply_chain_dqn")
obs = vec_env.reset()

In [27]:
for _ in range(20):
    action, _states = model.predict(obs)
    obs, reward, done, info = vec_env.step(action)
    vec_env.render()
    if done:
        obs, _ = vec_env.reset()


/usr/local/lib/python3.11/dist-packages/stable_baselines3/common/vec_env/base_vec_env.py:244: UserWarning: You tried to call render() but no `render_mode` was passed to the env constructor.
  warnings.warn("You tried to call render() but no `render_mode` was passed to the env constructor.")
